In [22]:
import math
import pickle
import numpy as np
import pandas as pd
import pickle as pkl
from scipy.stats import beta
from tqdm import tqdm
from collections import defaultdict

from transformers import BertTokenizer, BertModel

import os
import ast
#from Content import *
#from Venue import *

import torch

In [5]:
## People
## Time
## Topic
## Media Contents


In [10]:
out_dir = './data'
journalist = 'aliceysu'
with open(os.path.join(out_dir, f'{journalist}_dict.pkl'), 'rb') as f:
    data = pickle.load(f)

with open(os.path.join(out_dir, f'{journalist}_ids.pkl'), 'rb') as f:
    map_id = pickle.load(f)
    
with open(os.path.join(out_dir, f'{journalist}_lan.pkl'), 'rb') as f:
    map_lan = pickle.load(f)

with open(os.path.join(out_dir, f'{journalist}_type.pkl'), 'rb') as f:
    map_type = pickle.load(f)

with open(os.path.join(out_dir, f'{journalist}_reply.pkl'), 'rb') as f:
    map_reply = pickle.load(f)

alice = pd.DataFrame.from_dict(data)
alice_sort = alice.sort_values(by=['created_at'])
conv = pd.read_csv(os.path.join(out_dir, f'{journalist}_conv_labels.csv'))

data = pkl.load(open(os.path.join(out_dir, f'{journalist}_dict.pkl'), 'rb'))
journal = pd.DataFrame.from_dict(data)

In [258]:
contexts = []
for item in conv['context']:
    temp_s = ''
    if item == '0':
        contexts.append('Unified Twitter Taxonomy')

    else:
        for i in ast.literal_eval(item):
            temp_s  += (i['domain']['name'] + ' ' + i['entity']['name'])
        contexts.append(temp_s)
    
annotations = []
for item in conv['annotations']:
    tem_s = ''
    if item == '0':
        annotations.append('Others')

    else:
        for i in ast.literal_eval(item):
            temp_s += i['type'] + ' ' + i['normalized_text']

        annotations.append(temp_s)

In [264]:
journal['context'] = contexts
journal['annotations'] = annotations
journal_sort = journal.sort_values(by=['created_at'])

In [306]:
## topics
## people
ref_ids = []

journal_sort['topics'] = None
journal_sort['ppls'] = None
conv_ids = set(journal_sort['conversation_id'])
for conv in list(conv_ids):
    test_d = journal_sort[journal_sort['conversation_id'] == conv]
    for index, item in test_d.iterrows():
        i = 0
        tweet_id = item['tweet_id']
        ref_id = item['reference_id']
        conv_id = item['conversation_id']
        topic1 = item['context']
        anno1 = item['annotations']
        if len(test_d) == 1:
            topic2 = 'Unified Twitter Taxonomy'
            anno2 = 'Others'
            journal_sort.at[index, 'topics']=[text1, text2]
            journal_sort.at[index, 'ppls']=[anno1, anno2]
            continue
        if ref_id not in ref_ids:
            if tweet_id == test_d.iloc[0]['tweet_id']:
                text2 = test_d[test_d['tweet_id']==test_d.iloc[1]['tweet_id']]['context'].item()
                anno2 = test_d[test_d['tweet_id']==test_d.iloc[1]['tweet_id']]['annotations'].item()
            else:
                text2 = test_d.iloc[i-1]['context']
                anno2 = test_d.iloc[i-1]['annotations']
        else:
            #print(tweet_id,ref_id)
            text2 = test_d[test_d['tweet_id']==ref_id]['context'].item()
            anno2 = test_d[test_d['tweet_id']==ref_id]['annotations'].item()

        ref_ids.append(tweet_id)
        
        journal_sort.at[index, 'topics']=[text1, text2]
        journal_sort.at[index, 'ppls']=[anno1, anno2]
        #print(text2, journal_sort.at[index, 'topics'])
        i += 1
        

In [307]:
journal_sort.to_csv(os.path.join(out_dir, f'{journalist}_context.csv'))

12

In [310]:
context_sims = []
anno_sims = []

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
for index, item in tqdm(journal_sort.iterrows()):
    # Tokenize input text and convert to tensor
    c1 = item['topics'][0]
    c2 = item['topics'][1]
    
    a1 = item['ppls'][0]
    a2 = item['ppls'][1]
    inputs1 = tokenizer(c1, return_tensors="pt")
    inputs2 = tokenizer(c2, return_tensors="pt")

    # Generate embeddings
    with torch.no_grad():
        outputs1 = model(**inputs1)
        outputs2 = model(**inputs2)

    # The last hidden state is the sequence of hidden states of the last layer of the model
    last_hidden_states1 = outputs1.last_hidden_state
    last_hidden_states2 = outputs2.last_hidden_state

    # Optionally, use the [CLS] token's embedding as the representation for the entire sentence
    sentence_embedding1 = last_hidden_states1[:, 0, :]
    sentence_embedding2 = last_hidden_states2[:, 0, :]

    cosine = np.dot(sentence_embedding1,sentence_embedding2.T)/(norm(sentence_embedding1)*norm(sentence_embedding2))
    context_sims.append(cosine)
    #print(sentence_embedding.shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
10632it [17:00, 10.41it/s]


In [314]:
journal_sort['topics_sim'] = context_sims

In [315]:
journal_sort.to_csv(os.path.join(out_dir, f'{journalist}_context.csv'))

In [53]:
from numpy.linalg import norm
 
# define two lists or array
A = other_vec
B = sentence_embedding
 
# compute cosine similarity
cosine = np.dot(A,B.T)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

Cosine Similarity: [[0.9301709]]


In [49]:
np.dot(A,B.T)

array([[182.91821]], dtype=float32)

In [45]:
Organization_vec = sentence_embedding

In [40]:
other_vec = sentence_embedding

In [43]:
other_place_vec = sentence_embedding

In [ ]:
## PA * Latest * Field * Self = 2*2*2*2 = 16
## PA: Normal PA (p=0.1, p/#nodes + (1-p)indegree/sum_of_indegree), Uniform (1/#nodes)
## Latest: Normal Latest (beta(10,1), x=1-(outyear-inyear)/(outyear-oldest_year)), Uniform (1/(outyear-oldest_year))
## Field: Similar (1-(1-e^(-||x-y||_2))/(1-e^(-2)), x&y L2-normalized), Different ((1-e^(-||x-y||_2))/(1-e^(-2)), x&y L2-normalized)
## Self: Prefer (coauthors: 0.9/#coauthors, non-coauthors: 0.1/#non-coauthors), Not Prefer (coauthors: 0.1/#coauthors, non-coauthors: 0.9/#non-coauthors)

## 1st: Normal_PA * Normal_Latest * Similar_Field * Prefer_Self
## 2nd: Normal_PA * Normal_Latest * Similar_Field * NotPrefer_Self
## 3rd: Normal_PA * Normal_Latest * Different_Field * Prefer_Self
## 4th: Normal_PA * Normal_Latest * Different_Field * NotPrefer_Self
## 5th: Normal_PA * Uniform_Latest * Similar_Field * Prefer_Self
## 6th: Normal_PA * Uniform_Latest * Similar_Field * NotPrefer_Self
## 7th: Normal_PA * Uniform_Latest * Different_Field * Prefer_Self
## 8th: Normal_PA * Uniform_Latest * Different_Field * NotPrefer_Self
## 9th: Uniform_PA * Normal_Latest * Similar_Field * Prefer_Self
## 10th: Uniform_PA * Normal_Latest * Similar_Field * NotPrefer_Self
## 11th: Uniform_PA * Normal_Latest * Different_Field * Prefer_Self
## 12th: Uniform_PA * Normal_Latest * Different_Field * NotPrefer_Self
## 13th: Uniform_PA * Uniform_Latest * Similar_Field * Prefer_Self
## 14th: Uniform_PA * Uniform_Latest * Similar_Field * NotPrefer_Self
## 15th: Uniform_PA * Uniform_Latest * Different_Field * Prefer_Self
## 16th: Uniform_PA * Uniform_Latest * Different_Field * NotPrefer_Self

In [2]:
oldest = 1979
superbeta = beta(a=10,b=1)
superbeta_dist = np.array([superbeta.pdf((intime-oldest)/(outtime-oldest)) for outtime in range(2000,2019) for intime in range(1980,2018)]).reshape(2019-2000,2018-1980)


In [4]:
superbeta_dist.shape

(19, 38)

In [ ]:
superbeta = beta(a=10,b=1)
superbeta_dist = np.array([superbeta.pdf((intime-oldest)/(outtime-oldest)) for outtime in range(2000,2019) for intime in range(1980,2018)]).reshape(2019-2000,2018-1980)

def cal_cite_edgeprobs(outcontent):
    
    outyear, outfield, outcitations = outcontent.year, content_fields[outcontent.id], outcontent.outcitations
    outcocontents = set()
    for author,_,_ in outcontent.authors:
        for year in author_contents[author].keys():
            if year < outyear:
                for outcocontent in author_contents[author][year]:
                    outcocontents.add(outcocontent)

    edgeprobs = []
    for incontent in outcitations:
         
        pnormal_pa = 0.1/content_cumcounts[outyear-1]+0.9*content_eachdgs[incontent][outyear-1]/content_sumdgs[outyear-1]
        puniform_pa = 1/content_cumcounts[outyear-1]

        pnormal_latest = superbeta_dist[outyear-2000, content_year[incontent]-1980]
        puniform_latest = 1/(outyear-oldest)

        psim_field = 1-(1-math.exp(-np.linalg.norm(outfield-content_fields[incontent],2)))/(1-math.exp(-2))
        pdif_field = 1-psim_field
        
        ppre_self = 0.9/len(outcocontents) if incontent in outcocontents else 0.1/(content_cumcounts[outyear-1]-len(outcocontents))
        pnot_self = 0.1/len(outcocontents) if incontent in outcocontents else 0.9/(content_cumcounts[outyear-1]-len(outcocontents))
        
        temp1 = np.outer([pnormal_pa,puniform_pa],[pnormal_latest,puniform_latest]).flatten()
        temp2 = np.outer([psim_field,pdif_field],[ppre_self,pnot_self]).flatten()
        edgeprobs.append(np.outer(temp1,temp2).flatten())
    
    return np.array(edgeprobs, dtype=np.float32)